[Accueil](../../index.ipynb) > [Sommaire de Terminale](../index.ipynb)

# 3.4 Base de données - Langage SQL

Le langage SQL est un langage **déclaratif**, c'est à dire qu'il consiste à décrire les propriétés d'arrivée (le résultat) en fonction des données de départ (les données de la base) sans préciser la façon dont le résultat est calculé.

Le langage SQL a été normalisé en 1987 sous le nom de ISO/CEI 9075.

Sommaire de ce cours:

- [3.4.1 Interrogation de la base](#3.4.1)
- [3.4.2 Mise à jour, insertion et suppression](#3.4.2)
- [3.4.3 Création de tables (hors programme)](#3.4.3)

<div id="3.4.1"></div>

## 3.4.1 Interrogation de la base

Pour commencer l'apprentissage du langage SQL nous allons partir d'une base de données existante et utiliser un SGBD nommé **SQLite** ainsi que l'interface graphique **DB Browser for SQLite**.

- Dans votre répertoire **Projets**, créez les dossiers **SQL/TP1**. 
- Téléchargez dans le dossier TP1 [ce fichier zip](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip) et l'extraire.
- Lancez le logiciel **DB Browser for SQLite** et ouvrir le fichier **chinook.db**.

Voici le schéma de la base sur laquelle nous allons nous entrainer.

![database](img/sqlite-sample-database-color.jpg)

Nous n'utiliserons pas l'integralité des relations de la base. La cardinalité n-n entre **playlists** et **tracks** ne sera pas étudiée.

### Un SELECT sur une simple table

voici la **syntaxe minimale** pour faire une requête d'interrogation
``` sql
SELECT attribute FROM tablename;
```
<div class="alert alert-info">Toute requête SQL se termine par un point-virgule. Ici le logiciel l'ajoute si il est manquant.</div>


**Exemple** : Si on veut obtenir tous les titres de la relation albums on effectue donc:

``` sql
SELECT Title FROM albums;
```
On peut selectionner **plusieurs attributs** en les séparant par des virgules.

``` sql
SELECT attribute1, attribute2 FROM tablename;
```

**Exemple** : Obtenir tous les id et titres de la table albums :

``` sql
SELECT Title, AlbumId FROM albums;
```
On peut selectionner **tous les attributs** avec le quantificateur *.

``` sql
SELECT * FROM tablename;
```
<div class="alert alert-info">En dehors de tests, il est fortement déconseillé d'utiliser ce quantificateur. En effet si le nombre d'attributs venait à changer (ajout ou suppression) le nombre de colonnes changerait également. De plus il est rare de vouloir obtenir l'ensemble des attributs d'une table.</div>

#### Ajout de conditions

Il est possible d'interroger notre base en ajoutant le mot clé **WHERE**.

La syntaxe générale est :

``` sql
SELECT
	column_list
FROM
	table
WHERE
	search_condition;
```

**Exemple** : Obtenir le titre du film dont l'id est 10:

``` sql
SELECT Title FROM albums WHERE AlbumId = 10;
```

Les conditions de recherche permettent d'utiliser différents opérateurs:

- opérateurs de comparaison
  - =
  - <>   (différent)
  - <
  - \>
  - <=
  - \>=
- opérateurs logiques
  - AND
  - OR
  - [BETWEEN](https://www.sqlitetutorial.net/sqlite-between/)
  - [IN](https://www.sqlitetutorial.net/sqlite-in/)
  - [LIKE](https://www.sqlitetutorial.net/sqlite-like/)
  - NOT

**TP1 / Exercice 1**

- Trouver tous les morceaux de musique dont l'id de l'album est 1;
- Trouver tous les prénoms des clients qui commencent par 'E';
- Trouver les noms de familles des clients qui sont au USA et dont le prénom est Frank;
- Trouver les mails des clients qui sont au USA ou au Portugal;
- Trouver les titres des morceaux de musique qui durent plus de 15 minutes;

#### Obtenir des résultats sans les doublons

Comment obtenir tous les pays des clients ?

La requête suivante me retourne tous les pays de l'ensemble des clients

``` sql
SELECT Country FROM customers;
```
mais il y a beaucoup de doublons...

la clause **DISTINCT** permet de supprimer les doublons

``` sql
SELECT DISTINCT Country FROM customers;
```
Voici les règles à respecter:

 - DISTINCT doit être placé juste après le SELECT
 - il est suivi d'une ou plusieurs attributs

#### Ordonner les résultats

SQLite ne range pas les données des tables dans un ordre connu : l'ordre n'est pas celui de l'insertion. Il en résulte que les résultats obtenus peuvent sembler être rangés dans un ordre aléatoire.

La clause ORDER BY permet de ranger ses résultats. Voici la syntaxe générale:
``` sql
SELECT
   select_list
FROM
   table
WHERE
    search_condition
ORDER BY
    column_1 ASC,
    column_2 DESC;```

**Remarques** : 

- par défaut l'ordre est ASC (Croissant)
- comme le précise la syntaxe il est possible de spécifier plusieurs tris, dans ce cas le premier tri est effectué puis le deuxième...
- on peut trier selon une colonne qui n'est pas dans les colonnes du SELECT

**TP1/Exercice 2**

- Retourner tous les compositeurs et le nom des morceaux de musique dont le compositeur n'est pas NULL triés par compositeur;
- Retourner tous les compositeurs et le nom des morceaux de musique dont le compositeur n'est pas NULL triés par compositeur puis par nom;
- Retourner tous les noms, prénoms, les pays, les villes, des clients classés par pays, puis par ville puis par nom puis pas prénom.
- Retourner les morceaux de musique de l'album 1 triés par durée décroissante.

#### Limiter le nombre de résultats

La clause LIMIT est optionnelle dans le SELECT. Elle permet de limiter le nombre de résultats.

La syntaxe est la suivante:

``` sql
SELECT
	column_list
FROM
	table
LIMIT row_count;
```
**Exemple**

Retourner les dix premiers résultat des morceaux de musique

``` sql
SELECT trackId, name FROM tracks LIMIT 10;
```

Il est possible de déclarer le décalage souhaité avec la clause OFFSET optionnelle.

``` sql
SELECT trackId, name FROM tracks LIMIT 10 OFFSET 30;
```
**Remarque**

Cete option est très souvent utilisée dans les sites web pour paginer le nombre de résultats d'une requête ceci afin d'éviter d'avoir trop de résultats sur une même page.

**TP1/Exercice 3**

Reprendre l'exercice 3 en ajoutant les clauses LIMIT et OFFSET


#### Quelques fonctions d'aggregation

##### [COUNT](https://www.sqlitetutorial.net/sqlite-count-function/)

Voici sa syntaxe

``` sql
COUNT([ALL | DISTINCT] expression);
```
  - Si ALL est spécifié, count compte toutes les valeurs non nulles, doublons inclus. (C'est l'option par défaut)
  - Si DISTINCT est spécifié, count compte uniquement les valeurs uniques non nulles

##### [SUM](https://www.sqlitetutorial.net/sqlite-sum/)

Voici sa syntaxe

``` sql
SUM([ALL | DISTINCT] expression);
```
- Si ALL est spécifié, sum ajoute toutes les valeurs non nulles, doublons inclus. (C'est l'option par défaut)
- Si DISTINCT est spécifié, sum ajoute uniquement les valeurs uniques non nulles

##### [MIN](https://www.sqlitetutorial.net/sqlite-sum/) / [MAX](https://www.sqlitetutorial.net/sqlite-max/)

Voici leurs syntaxes

``` sql
MIN(expression);
```

``` sql
MAX(expression);
```

##### [AVG](https://www.sqlitetutorial.net/sqlite-avg/)

La fonction AVG retourne la valeur moyenne de valeurs non nulles

Voici sa syntaxe

``` sql
AVG([ALL | DISTINCT] expression);
```

Par défaut la fonction *AVG* utilise la clause *ALL* par défaut. Si vous désirez déterminer la moyenne de valeurs distinctes, il faut spécifier la clause *DISTINCT*.

**TP1/Exercice 4**

- Donner le nombre de morceaux de l'album dont l'id est 1;
- Donner le nombre de pays différents où habitent les clients;
- Donner la durée total (en heures) de l'ensemble des morceaux de musique

**Remarque**

- Si le diviseur ET le dividende sont entiers, le quotient est entier;
- Si le diviseur OU le dividende est un float, le quotient est un float.

https://isn-icn-ljm.pagesperso-orange.fr/SQL/

### Jointure de TABLES

Pour l'instant nous avons effectué des interrogations sur notre base en utilisant **une seule table** par requête.

Il serait tout de même intéressant de pouvoir lister les titres, noms d'albums et nom de l'artiste d'une playlist.

C'est ce que nous allons apprendre à faire avec les **jointures de tables**.

<div class="alert alert-info">Il existe plusieurs types de jointures de tables. Cette année nous n'en étudierons qu'une seule : la INNER JOIN</div>

#### Sur deux tables

Nous allons nous interesser à cette partie de notre base:

![](img/tracks_albums.png)

La requete à executer est  la suivante:

``` sql
SELECT tracks.Name, albums.Title
FROM tracks
INNER JOIN albums ON albums.Albumid = tracks.Albumid;
```

**FROM tracks INNER JOIN albums** : on joint les deux tables

**ON albums.Albumid = tracks.Albumid** : on ajoute comme condition que la clé primaire d'un album doit être égale à la clé étrangère AlbumId de tracks

<div class="alert alert-info">ici nous utilisons plusieurs tables : afin de limiter tout ambiguité, on préciser le <b>nom de la table . son attribut</b></div>

**A FAIRE:**

- Vérifier que cela fonctionne en la faisant uniquement sur l'album 1. Voir https://fr.wikipedia.org/wiki/For_Those_About_to_Rock_We_Salute_You
- Que se passe t-il si on enleve le ON albums.Albumid = tracks.Albumid ?

Les noms de colonnes ne sont pas très explicites.
![](img/captureRequete1.png)

Corrigeons cela.

``` sql
SELECT tracks.Name AS Titre, albums.Title as Album
FROM tracks
INNER JOIN albums ON albums.Albumid = tracks.Albumid
WHERE tracks.Albumid=1;
```
#### Sur trois tables

![](img/artists_albums_tracks.png)

On aimerait afficher les infos précédentes + le nom du groupe de l'album pour le groupe d'id 1

C'est en fait assez simple:

``` sql
SELECT artists.Name, albums.Title, tracks.Name, 
FROM tracks
INNER JOIN albums ON albums.Albumid = tracks.Albumid
INNER JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.ArtistId = 1;
```

Bon, à vous de jouer...

<div id="3.4.2"></div>

## 3.4.2 Mise à jour, insertion et suppression

### Modification de ligne avec UPDATE

Pour mettre à jour des lignes on utilise la commande **UPDATE**.

La syntaxe générale est la suivante

```sql
UPDATE table
SET column_1 = new_value_1,
    column_2 = new_value_2
WHERE
    search_condition 
```

**Exemple**

Modifier tous les morceaux de musique qui sont de genre 'Rock and roll' (id=5) en genre 'Rock' (id=1)

```sql
UPDATE tracks SET GenreId=1  WHERE GenreId=5;
```

### Ajout de ligne avec INSERT

Afin d'ajouter des lignes dans une table on utilise **INSERT**.

La syntaxe générale est la suivante:

```sql
INSERT INTO table[columns] VALUES(...);
```

**Exemple**

- Ajouter un nouvel artiste dans  la table Artists

```sql
INSERT INTO Artists(Name)
VALUES ("Didier Super");
```
On remarque qu'il n'est pas nécessaire de spécifier l'identifiant, celui-ci est auto incrémenté.

En revanche cette requête échoue:

```sql
INSERT INTO Artists
VALUES (1,"Didier Super");
```

```
Result: UNIQUE constraint failed: artists.ArtistId
At line 1:
INSERT INTO Artists
VALUES (1,"Didier Super");
```
La contrainte sur l'unicité de la clé primaire provoque un échec.

### Suppression de ligne avec DELETE

Il est possible de supprimer plusieurs lignes avec la commmande **DELETE**.

La syntaxe générale est la suivante:

```sql
DELETE FROM table
WHERE search_condition;
```

**Exemple**

Je n'en ai pas trouvé...
La suppression des lignes est interdite par les contraintes sur la clé étrangère.

C'est ce qui arrive souvent...

par exemple, supprimer tous les morceaux de l'album black sabbah (id=12).

```sql
DELETE from tracks WHERE AlbumId=12;
```

```
Result: FOREIGN KEY constraint failed
At line 1:
DELETE FROM tracks WHERE AlbumId=12;
```
En effet ces morceaux se trouvent peut-etre dans une playlist ou une facture.


La modification / insertion /suppression sont sujettes aux contraintes:

<div class="info alert-info">
Voici le respect des contraintes d'intégrité
    <ul>
        <li>Une clé primaire doit être unique et non NULL;</li>
        <li>On ne peut pas insérer une ligne avec une clé primaire qui existe déjà;</li>
        <li>On ne peut pas modifier la valeur d’une clé primaire en une autre valeur qui existe déjà;</li>
        <li>Une clé étrangère doit référencer une clé primaire existante;</li>
        <li>Il faut créer la ligne contenant la clé primaire avant une ligne contenant une clé étrangère la référençant;</li>
        <li>On ne peut pas modifier une clé primaire si elle est déjà référencée;</li>
        <li>On ne peut pas effacer une ligne contenant une clé primaire déjà référencée;</li>
        <li>Il est possible de mettre des contraintes supplémentaires sur les clés pour gérer les cascades de modifications. (voir <a href="(https://www.sqlitetutorial.net/sqlite-foreign-key/">ici</a> et rechercher CASCADE)</li>
    </ul>
</div>

<div id="3.4.3"></div>

## 3.4.3 Création de tables (hors programme)

Reprenons l'exemple du prof-classe

![](img/schemaRelationnel3.png)

### Création de la relation professeur

``` sql

CREATE TABLE professeur (
	numen INTEGER PRIMARY KEY,
	prenom TEXT NOT NULL,
	nom TEXT NOT NULL,
);
```

Le code SQL parle de lui même...

### Création de la relation classe

``` sql

CREATE TABLE classe (
	id_classe INTEGER PRIMARY KEY,
	label TEXT NOT NULL,
);

```

Aucune surprise...

### Création de la relation prof_classe

Là c'est un peu plus technique puisqu'il faut déclarer, la clé primaire par un couple d'attributs **ET** les clés étrangères.

``` sql

CREATE TABLE prof_classe(
   numen INTEGER,
   id_classe INTEGER,
   PRIMARY KEY (numen, id_classe),
   FOREIGN KEY (numen) 
      REFERENCES professeur (numen) 
   FOREIGN KEY (id_classe) 
      REFERENCES classe (id_classe) 
);
```

On voit comment on crée une clé primaire (obligatoire dans toute relation) à partir du couple (numen, id_classe).
Chaque clé étrangère est définie par

- un nom
- une référence vers la clé primaire de la table liée

Voici les liens qui concernent les créations/modification/suppression de table dans SQLite

- [Creation de table](https://www.sqlitetutorial.net/sqlite-create-table/)
- [Modification de table](https://www.sqlitetutorial.net/sqlite-alter-table/)
- [Suppression de table](https://www.sqlitetutorial.net/sqlite-drop-table/)
- [clé primaire](https://www.sqlitetutorial.net/sqlite-primary-key/)
- [clé étrangère](https://www.sqlitetutorial.net/sqlite-foreign-key/)

## Injection de code SQL

L'injection de code SQL est un type d'attaque qui permet de modifier une base de données en injectant du code SQL en utilisant une entrée non préue à cet effet, par exemple un formulaire web.

Voici le cartoon [XKCD](https://xkcd.com/) qui met en garde sur l'injection de code.

![](img/exploits_of_a_mom.png)

Une blague de geek:

![](img/tablice.png)


ainsi qu'un lien vers l'[injection de code SQL](https://fr.wikipedia.org/wiki/Injection_SQL).

## Exercices supplémentaires

- [Une base de maternité : bac 2022 Amérique du sud](ex_sql_maternite.ipynb)

[Accueil](../../index.ipynb) > [Sommaire de Terminale](../index.ipynb)